In [1]:
!pip install pandas
!pip install requests
!pip install -U matplotlib
!pip install seaborn
!pip install BeautifulSoup4
!pip install psycopg2

    100% |████████████████████████████████| 10.4MB 2.1MB/s eta 0:00:01   15% |████▉                           | 1.6MB 27.2MB/s eta 0:00:01
    100% |████████████████████████████████| 20.0MB 584kB/s eta 0:00:01   34% |███████████                     | 6.9MB 24.6MB/s eta 0:00:01
    100% |████████████████████████████████| 512kB 26.8MB/s ta 0:00:01
    100% |████████████████████████████████| 13.1MB 1.2MB/s eta 0:00:01  7% |██▍                             | 972kB 14.8MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 9.9MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 17.4MB/s ta 0:00:01
    100% |████████████████████████████████| 215kB 6.4MB/s eta 0:00:01
    100% |████████████████████████████████| 25.2MB 722kB/s eta 0:00:01  1% |▍                               | 317kB 13.3MB/s eta 0:00:02    3% |█                               | 860kB 24.8MB/s eta 0:00:01    13% |████▌                           | 3.5MB 14.0MB/s eta 0:00:02
    100% |███████████████████████

In [1]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

In [14]:
try:
    conn = psycopg2.connect("dbname='postgres' user='postgres' host='db-zminj1.czg4cayqnvyt.us-east-2.rds.amazonaws.com' password='Args072003'")
except:
    print("I am unable to connect to the database")

In [15]:
competicao = 'Campeonato Brasileiro - Serie A'
temporada = '2019'
cursor = conn.cursor()  
cursor.execute("SELECT link_partida FROM public.tb_partidas WHERE competicao = %s and temporada = %s;",(competicao,temporada))  
rows = cursor.fetchall()

In [16]:
for x in rows:
    array_url = str(x[0]).split('/')
    partida = array_url[len(array_url) - 1]
    print('Consultando partida ' + partida + '...')
    req = requests.get(str(x[0]))
    if req.status_code == 200:
        print('Requisição bem sucedida!')
        content = req.content
        soup = BeautifulSoup(content, 'html.parser')
        data_tables = soup.find_all(name='table')  # Consulta varios elementos com a tag table
        i = 0
        nome_equipe_casa = ''
        nome_equipe_visitante = ''
        formacao_equipe_casa = ''
        formacao_equipe_visitante = ''
        while i < len(data_tables):
            
            for row in data_tables[0].select("tr"): # Consulta nome e formacao da equipe da casa na tabela 0
                cellsTh = row.findAll("th")
                if(len(cellsTh) > 0 and str(cellsTh[0].string) != 'Banco'):
                    nome_clube_bruto = str(cellsTh[0].string)
                    array_nome = nome_clube_bruto.split(' ')
                    if(len(array_nome) == 2):
                        nome_equipe_casa = array_nome[0]
                        formacao_equipe_casa = array_nome[1]
                    elif(len(array_nome) == 3):
                        nome_equipe_casa = array_nome[0] + ' ' + array_nome[1]
                        formacao_equipe_casa = array_nome[2]
                    elif(len(array_nome) == 4):
                        nome_equipe_casa = array_nome[0] + ' ' + array_nome[1] + ' ' + array_nome[2]
                        formacao_equipe_casa = array_nome[3]
            
            for row in data_tables[1].select("tr"): # Consulta nome e formacao da equipe  visitante na tabela 1
                cellsTh = row.findAll("th")
                if(len(cellsTh) > 0 and str(cellsTh[0].string) != 'Banco'):
                    nome_clube_bruto = str(cellsTh[0].string)
                    array_nome = nome_clube_bruto.split(' ')
                    if(len(array_nome) == 2):
                        nome_equipe_visitante = array_nome[0]
                        formacao_equipe_visitante = array_nome[1]
                    elif(len(array_nome) == 3):
                        nome_equipe_visitante = array_nome[0] + ' ' + array_nome[1]
                        formacao_equipe_visitante = array_nome[2]
                    elif(len(array_nome) == 4):
                        nome_equipe_visitante = array_nome[0] + ' ' + array_nome[1] + ' ' + array_nome[2]
                        formacao_equipe_visitante = array_nome[3]
                    
            if(i >= 3): # a partir da terceira tabela são as tabelas de dados estatisticos dos jogadores
                for row in data_tables[i].select("tr"): # Verifica tr
                    name_player = '' # Variavel com o nome do jogador
                    cellsTh = row.findAll("th") # Verifica th para capturar nome dos jogadores que nao estao em td
                    for rowCellTh in cellsTh: # percorre th para filtrar apenas com o atributo data-stat = player
                        if(rowCellTh['data-stat'] and rowCellTh['data-stat'] == 'player'):
                            name_player_tag = row.findAll("a")
                            if(len(name_player_tag) > 0):
                                name_player = str(name_player_tag[0].string)
                    cells = row.findAll("td")                    
                    if(len(cells) == 15): # Captura estatisticas de jogadores
                        equipe = nome_equipe_casa if i == 3 else nome_equipe_visitante
                        formacao = formacao_equipe_casa if i == 3 else formacao_equipe_visitante
                        numero_camisa =  str(cells[0].string)
                        pais_a = cells[1].findAll("a") # captura tag a da celula 1 para capturar span
                        pais_span = pais_a[0].findAll("span") # captura tag span da celula 1 para capturar string
                        pais_span_array = str(pais_span[0]).split(' ') # captura array de span encontrados
                        pais = pais_span_array[5][0:3] # seleciona pais do item 5 do array de span
                        posicao =  str(cells[2].string)
                        minutos =  str(cells[3].string) if type(cells[3].string) != 'NoneType' else 0
                        gols =  str(cells[4].string) if type(cells[4].string) != 'NoneType' else 0
                        assistencias = str(cells[5].string) if type(cells[5].string) != 'NoneType' else 0
                        total_chutes = str(cells[6].string) if type(cells[6].string) != 'NoneType' else 0
                        chutes_a_gol = str(cells[7].string) if type(cells[7].string) != 'NoneType' else 0
                        faltas_cometidas = str(cells[8].string) if type(cells[8].string) != 'NoneType' else 0
                        faltas_provocadas = str(cells[9].string) if type(cells[9].string) != 'NoneType' else 0
                        cruzamentos = str(cells[10].string) if type(cells[10].string) != 'NoneType' else 0
                        divididas_vencidas = str(cells[11].string) if type(cells[11].string) != 'NoneType' else 0
                        cortes = str(cells[12].string) if type(cells[12].string) != 'NoneType' else 0
                        cartoes_amarelos = str(cells[13].string) if type(cells[13].string) != 'NoneType' else 0
                        cartoes_vermelhos = str(cells[14].string) if type(cells[14].string) != 'NoneType' else 0
                        
                        try: # Verifica se a partida já foi salva anteriormente
                            print('Consultando partida na base de dados...')
                            link_partida = str(x[0])
                            cursor.execute("SELECT COUNT(*) FROM public.tb_stats_jogadores WHERE link_partida = %s AND nome_jogador = %s AND numero_camisa = %s;", (link_partida, name_player, numero_camisa))
                            rows_c = cursor.fetchall()
                            if (rows_c[0][0] >= 1):
                                print('Partida salva anteriormente')
                            else: # Caso a partida nao tenha sido salva anteriormente tenta salvar
                                try:
                                    cursor.execute("INSERT INTO public.tb_stats_jogadores (competicao, temporada, nome_jogador, equipe_jogador, formacao, numero_camisa, pais, posicao, minutos, gols, assistencias, total_chutes, chutes_a_gol, faltas_cometidas, faltas_provocadas, cruzamentos, divididas_vencidas, cortes, cartoes_amarelos, cartoes_vermelhos, link_partida) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                                        (competicao, temporada, name_player, equipe, formacao, numero_camisa, pais, posicao, minutos, gols, assistencias,
                                         total_chutes, chutes_a_gol, faltas_cometidas, faltas_provocadas, cruzamentos, divididas_vencidas, cortes, cartoes_amarelos, cartoes_vermelhos, link_partida))
                                    print('Partida do jogador ' + name_player + ' salva com sucesso!')
                                except Exception as ex:
                                    print('Erro ao salvar partida do jogador ' + name_player + ' erro: ' + str(ex))
                        except Exception as ex:
                            print('Erro ao consultar partida - erro: ' + str(ex))
    #                         print(equipe + ' - ' + formacao + ' - ' + name_player + '-' + numero_camisa + '-' + pais + '-' + posicao + '-' + minutos + '-' + gols + '-' + assistencias + '-' + total_chutes)
            i += 1

Consultando partida Sao-Paulo-Botafogo-RJ-2019Abril27-Serie-A...
Requisição bem sucedida!
Consultando partida na base de dados...
0
Partida do jogador Alexandre Pato salva com sucesso!
Consultando partida na base de dados...
0
Partida do jogador Jonas salva com sucesso!
Consultando partida na base de dados...
0
Partida do jogador Éverton salva com sucesso!
Consultando partida na base de dados...
0
Partida do jogador Igor Gomes salva com sucesso!
Consultando partida na base de dados...
0
Partida do jogador Hernanes salva com sucesso!
Consultando partida na base de dados...
0
Partida do jogador Antony salva com sucesso!
Consultando partida na base de dados...
0
Partida do jogador Hudson salva com sucesso!
Consultando partida na base de dados...
0
Partida do jogador Tchê Tchê salva com sucesso!
Consultando partida na base de dados...
0
Partida do jogador Vitor Bueno salva com sucesso!
Consultando partida na base de dados...
0
Partida do jogador Reinaldo salva com sucesso!
Consultando part

In [17]:
conn.commit()
conn.close()